Começaremos o projeto aplicando o índice de Jaccard e o filtro TPEI nos dados iniciais.
O filtro TPEI excluirá grande parte dos dados desnecessários pois a equivalência só será válida para uma quantidade de TPEI igual ou maior. O índice de Jaccard analisará os dados a partir de um grafo direcionado baseado nas recomendações de uma disciplina. Sendo assim, é necessário criar esse grafo


Bibliotecas

In [ ]:
import pandas as pd
import networkx as nx
import unicodedata
import requests
from io import StringIO
from node2vec import Node2Vec
import numpy as np

Preparar os dados:

In [5]:
# ✅ Normaliza uma string removendo acentos e espaços extras
def normalize_str(s: str) -> str:
    return (
        unicodedata.normalize('NFKD', s)
        .encode('ASCII', 'ignore')
        .decode('utf-8')
        .strip()
    )

# ✅ Versão específica para nomes de disciplinas (já normaliza e coloca lowercase)
def normalizar_nome(nome: str) -> str:
    return normalize_str(nome).lower()

# ✅ Baixa o catálogo da UFABC direto do GitHub
def carregar_catalogo():
    url = "https://raw.githubusercontent.com/angeloodr/disciplinas-ufabc/main/catalogo_disciplinas_graduacao_2024_2025.tsv"
    print("🔄 Baixando catálogo de disciplinas...")
    resp = requests.get(url)
    resp.raise_for_status()
    df = pd.read_csv(StringIO(resp.text), sep='\t')

    # Normaliza os nomes das colunas
    df.columns = [
        normalize_str(col).upper().replace(' ', '_')
        for col in df.columns
    ]
    print("✅ Download bem-sucedido!")
    print("📝 Colunas disponíveis:", df.columns.tolist())
    return df


Construir o grafo

In [6]:
# ✅ Constrói grafo com arestas de pré-requisito com base na coluna RECOMENDACAO
def construir_grafo(df: pd.DataFrame) -> nx.DiGraph:
    print("📌 Construindo grafo de pré-requisitos...")
    G = nx.DiGraph()
    total_arestas = 0

    # Cria um dicionário de nome normalizado → sigla
    mapping = {
        normalizar_nome(row['DISCIPLINA']): row['SIGLA']
        for _, row in df.iterrows()
        if pd.notna(row['SIGLA'])
    }

    # Adiciona todos os nós com metadados
    for _, row in df.iterrows():
        sigla = row['SIGLA']
        nome = row['DISCIPLINA']
        if pd.isna(sigla): 
            continue
        G.add_node(sigla, nome=nome)

    # Adiciona as arestas baseadas nas recomendações
    for _, row in df.iterrows():
        curso = row['SIGLA']
        if pd.isna(curso):
            continue

        recs = row.get('RECOMENDACAO', '')
        if pd.isna(recs) or not isinstance(recs, str):
            continue

        for rec in recs.split(';'):
            rec = rec.strip()
            if not rec:
                continue
            rec_norm = normalizar_nome(rec)
            if rec_norm in mapping:
                prereq = mapping[rec_norm]
                if not G.has_edge(prereq, curso):
                    G.add_edge(prereq, curso, tipo='pre_requisito')
                    total_arestas += 1

    print(f"✅ Grafo criado com {G.number_of_nodes()} nós e {total_arestas} arestas.")
    return G


Configurar o grafo para rede direcionada e criar os embenddings estruturais

In [ ]:
# ✅ Remove ciclos do grafo para que seja um DAG (necessário para cálculo de profundidade)
def remover_ciclos(G: nx.DiGraph) -> nx.DiGraph:
    print("🔁 Removendo ciclos (modo eficiente)...")
    G_copy = G.copy()
    removidos = 0
    try:
        while True:
            ciclo = nx.find_cycle(G_copy, orientation='original')
            if ciclo:
                # Remove a primeira aresta do ciclo
                u, v, _ = ciclo[0]
                G_copy.remove_edge(u, v)
                removidos += 1
    except nx.NetworkXNoCycle:
        pass  # Não há mais ciclos

    print(f"✅ Ciclos removidos: {removidos}")
    return G_copy

# ✅ Gera embeddings estruturais com Node2Vec
def gerar_embeddings_node2vec(G: nx.DiGraph, dimensions=64, walk_length=10, num_walks=50, workers=1):
    print("🔄 Gerando embeddings estruturais (Node2Vec)...")
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)
    model = node2vec.fit(window=5, min_count=1, batch_words=4)
    embeddings = {node: model.wv[node] for node in G.nodes()}
    print("✅ Embeddings gerados!")
    return embeddings

# ✅ Calcula profundidade de cada nó no grafo (camada curricular)
def calcular_profundidade(G: nx.DiGraph):
    print("📏 Calculando profundidade no grafo (DAG)...")
    profundidades = {}
    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        if not preds:
            profundidades[node] = 0
        else:
            profundidades[node] = max(profundidades[p] for p in preds) + 1
    print("✅ Profundidade calculada!")
    return profundidades

# ✅ Salva os embeddings em CSV
def salvar_embeddings_csv(embeddings, caminho="embeddings_node2vec.csv"):
    df = pd.DataFrame.from_dict(embeddings, orient='index')
    df.index.name = 'DISCIPLINA'
    df.to_csv(caminho)
    print(f"💾 Embeddings salvos em: {caminho}")

# ✅ Fluxo principal
if __name__ == "__main__":
    # Etapa 1 - Carregar catálogo
    df = carregar_catalogo()

    # Etapa 2 - Construir grafo
    grafo = construir_grafo(df)
    nx.write_graphml(grafo, "grafo_pre_requisitos.graphml")
    print("📁 Arquivo salvo: grafo_pre_requisitos.graphml")

    # Etapa 3 - Gerar embeddings estruturais
    embeddings = gerar_embeddings_node2vec(grafo)
    salvar_embeddings_csv(embeddings)

    # Etapa 4 - Remover ciclos e calcular profundidade
    grafo_sem_ciclos = remover_ciclos(grafo)
    profundidades = calcular_profundidade(grafo_sem_ciclos)

    # Etapa 5 - Salvar profundidades em TXT
    with open("profundidade_nos.txt", "w") as f:
        for node, prof in profundidades.items():
            f.write(f"{node}: {prof}\n")
    print("📁 Profundidades salvas em: profundidade_nos.txt")

Com o grafo pronto e preparado para o uso ao executar os devidos carregamentos, começaremos a aplicação do Jaccard e outras similaridades:

In [8]:
# 🔢 Jaccard entre dois conjuntos
def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

# 🔁 Aplica Jaccard entre predecessores ou sucessores
def similaridade_jaccard(G, a, b, tipo='predecessor'):
    try:
        if tipo == 'predecessor':
            set_a = set(G.predecessors(a))
            set_b = set(G.predecessors(b))
        else:
            set_a = set(G.successors(a))
            set_b = set(G.successors(b))
        return jaccard_similarity(set_a, set_b)
    except nx.NetworkXError:
        return 0.0


Aplicar similaridade cosseno e de profundidade


In [9]:
# 📏 Similaridade de profundidade
def similaridade_profundidade(prof, a, b):
    if a not in prof or b not in prof:
        return 0.0
    max_p = max(prof.values())
    if max_p == 0:
        return 1.0
    return 1.0 - abs(prof[a] - prof[b]) / max_p

# 🧬 Similaridade por Node2Vec (cosseno)
def similaridade_node2vec(embeddings, a, b):
    if a not in embeddings.index or b not in embeddings.index:
        return 0.0
    vec_a = embeddings.loc[a].values.reshape(1, -1)
    vec_b = embeddings.loc[b].values.reshape(1, -1)
    return cosine_similarity(vec_a, vec_b)[0][0]


Combinar as diferentes similaridades

In [10]:
# 🧠 Combina múltiplas similaridades em score final
def similaridade_combinada(G, embeddings, profundidades, a, b, pesos=None):
    if pesos is None:
        pesos = {
            'jaccard_pred': 1.0,
            'jaccard_succ': 1.0,
            'profundidade': 1.0,
            'node2vec': 1.0,
        }

    sim_jaccard_pred = similaridade_jaccard(G, a, b, tipo='predecessor')
    sim_jaccard_succ = similaridade_jaccard(G, a, b, tipo='successor')
    sim_profundidade = similaridade_profundidade(profundidades, a, b)
    sim_node2vec = similaridade_node2vec(embeddings, a, b)

    total_peso = sum(pesos.values())
    score = (
        pesos['jaccard_pred'] * sim_jaccard_pred +
        pesos['jaccard_succ'] * sim_jaccard_succ +
        pesos['profundidade'] * sim_profundidade +
        pesos['node2vec'] * sim_node2vec
    ) / total_peso

    return {
        "score_combinado": score,
        "jaccard_pred": sim_jaccard_pred,
        "jaccard_succ": sim_jaccard_succ,
        "profundidade": sim_profundidade,
        "node2vec": sim_node2vec
    }

Filtro TPEI


In [11]:
# 🎯 Filtro TPEI exato: compara atributos T, P, E, I
def filtro_tpei_exato(G, a, b):
    for campo in ['T', 'P', 'E', 'I']:
        if G.nodes[a].get(campo) != G.nodes[b].get(campo):
            return False
    return True

Com isso, temos nossos dados preparados para a próxima etapa. O intuito é diminuir os dados de forma eficiente a cada etapa para reduzir o custo computacional final
